# Financial statement as waterfall chart

This notebook produces a waterfall chart showing different items of the company's financial statement. The model works with <b>public companies</b> only.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import warnings
import ipywidgets as widgets
import ipysheet
import refinitiv.data as rd
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f6a91ae3190 {name='codebook'}>

In [10]:
df = pd.DataFrame()
name = ''
datatable = None

ric = 'VOD.L'

fields = ['TR.CompanyName',
          'TR.F.TotRevBizActiv',
          'TR.F.ProvImpairForLoanLosses',
          'TR.F.CostOfOpRev',
          'TR.F.SGATot',
          'TR.F.OthOpExpnIncNet',
          'TR.F.OpProfBefNonRecurIncExpn',
          'TR.F.FinIncExpnNetTot',
          'TR.F.SaleOfTangIntangFixedAssetsGL',
          'TR.F.EqEarnLossBefTaxInclNonRecur',
          'TR.F.OthNonOpIncExpnTot',
          'TR.F.NonRecurIncExpnTot',
          'TR.F.IncBefTax',
          'TR.F.IncTax',
          'TR.F.EqInEarnLossOfAffilAfterTax',
          'TR.F.AfterTaxAdjOthTot',
          'TR.F.ExordActivAfterTaxGL',
          'TR.F.MinIntr',
          'TR.F.IntrExpnHybridDebtInstrEq',
          'TR.F.DistribForPrefShr',
          'TR.F.EarnAdjToNetIncOthExpnInc',
          'TR.F.IncAvailToComShr'
          ]

multipliers = [1, -1, -1, -1, -1, 1, 1, 1, 1,
               1, 1, 1, -1, 1, 1, 1, -1, -1, -1, -1, 1]

measure = ['absolute',
           'relative',
           'relative',
           'relative',
           'relative',
           'absolute',
           'relative',
           'relative',
           'relative',
           'relative',
           'relative',
           'absolute',
           'relative',
           'relative',
           'relative',
           'relative',
           'relative',
           'relative',
           'relative',
           'relative',
           'absolute'
           ]

category = ['Revenue',
            'Operating Expenses',
            'Operating Expenses',
            'Operating Expenses',
            'Operating Expenses',
            'Operating Profit',
            'Non-operating Expenses',
            'Non-operating Expenses',
            'Non-operating Expenses',
            'Non-operating Expenses',
            'Non-recurring Income/Expense',
            'Pre-tax Income',
            'Taxes',
            'After Tax Income/Expense',
            'After Tax Income/Expense',
            'After Tax Income/Expense',
            'After Tax Income/Expense',
            'Net Income',
            'Net Income',
            'Net Income',
            'Net Income'
            ]


def get_data(ric, fields):
    global df, name
    if ric != '':
        _df = rd.get_data(ric, fields)
        name = _df['Company Name'][0]
        _df = _df.drop(columns='Instrument')
        _df = _df.drop(columns='Company Name')
        df2 = pd.DataFrame(multipliers, columns=[1]).T
        df3 = pd.DataFrame(measure, columns=[2]).T
        df2.columns = _df.columns
        df3.columns = _df.columns
        _df = pd.concat([_df, df2, df3])
        _df = _df.dropna(axis=1)
        df = _df
    return


def update_plot(fig):
    fig.data = []
    trace = go.Waterfall(
        x=df.columns.tolist(),
        y=df.iloc[0]*df.iloc[1].tolist(),
        measure=df.iloc[2],
        increasing=dict(marker=dict(color='#3AC46D')),
        decreasing=dict(marker=dict(color='#F6465C')),
        totals=dict(marker=dict(color='#6978F7')),
        name=name
    )
    fig.update_layout(title=dict(text=name))
    fig.add_trace(trace)
    return


def update_table():
    global sheet, tab
    _df_to_table = pd.DataFrame()
    _df_to_table['Item'] = df.columns.tolist()
    _df_to_table['Latest'] = df.iloc[0].values.tolist()
    table = ipysheet.from_dataframe(_df_to_table)
    table.row_headers = False
    sheet = table
    tab.children = [wfig, sheet]
    return


def update_ric(value):
    global ric_input, ric
    value = value['new']
    if value != '':
        ric = value
        get_data(ric, fields)
        update_plot(wfig)
        update_table()
        ric_input.value = ''
    return


wfig = go.FigureWidget()

get_data(ric, fields)

trace = go.Waterfall(
    x=df.columns.tolist(),
    y=df.iloc[0]*df.iloc[1].tolist(),
    measure=df.iloc[2],
    increasing=dict(marker=dict(color='#3AC46D')),
    decreasing=dict(marker=dict(color='#F6465C')),
    totals=dict(marker=dict(color='#6978F7')),
    name=name
)

wfig.update_layout(
    margin=dict(t=5, b=5, l=5, r=5),
    plot_bgcolor='#1A1A1D',
    paper_bgcolor='#1A1A1D',
    yaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black', zerolinewidth=1,
               side='right', color='#D6D6D5', linecolor='#D6D6D5', ticks='outside'),
    xaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black',
               zerolinewidth=1, color='#D6D6D5', linecolor='#D6D6D5', ticks='outside'),
    colorway=['#6978F7', '#A325E9', '#96E05D', '#4A7FB9',
              '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'],
    legend=dict(font=dict(color='#D6D6D5'), orientation='h',
                yanchor='top', xanchor='center', y=1.05, x=0.1),
    title=dict(text=name, xanchor='center', yanchor='top',
               x=0.5, font=dict(color='#D6D6D5'))
)

wfig.add_trace(trace)

_df_to_table = pd.DataFrame()
_df_to_table['Item'] = df.columns.tolist()
_df_to_table['Latest'] = df.iloc[0].values.tolist()
sheet = ipysheet.from_dataframe(_df_to_table)
sheet.row_headers = False

tab = widgets.Tab()
tab.children = [wfig, sheet]
tab.set_title(0, 'Chart')
tab.set_title(1, 'Table')

ric_input = widgets.Text(description='RIC', continuous_update=False, style={
                         'description_width': '30px'})
ric_input.observe(update_ric, names='value')
widgets.VBox([ric_input, tab])

In [7]:
rd.close_session()